In [1]:
!pip install torch python-dotenv boto3 --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.4.0 requires botocore<1.27.60,>=1.27.59, but you have botocore 1.29.143 which is incompatible.


In [2]:
%load_ext dotenv
%dotenv env

In [3]:
!aws s3 ls s3://dissertation-data-dmiller

/usr/bin/sh: 1: aws: not found


In [4]:
import torch
import os
import argparse
import logging
import boto3
from pprint import pformat
from torch import nn
import io
import tqdm
import numpy as np
import pandas as pd

In [5]:
import os

LABEL = [
    "continue_work_session_30_minutes"
]

METADATA = [
    "user_id",
    "session_30_raw",
    "cum_platform_event_raw",
    "cum_platform_time_raw",
    "cum_session_time_raw",
    "global_events_user",
    "global_session_time",
    
    "year",
    "month",
    "day",
    "hour",
    "minute",
    "second"
]

OUT_FEATURE_COLUMNS = [
    "country_count", 
    "date_hour_sin", 
    "date_hour_cos",
    "date_minute_sin",
    "date_minute_cos",
    
    "session_30_count",
    "session_5_count",
    "cum_session_event_count",
    "delta_last_event",
    "cum_session_time",
    
    "expanding_click_average",
    "cum_platform_time",
    "cum_platform_events",
    "cum_projects",
    "average_event_time",
    
    "rolling_session_time",
    "rolling_session_events",
    "rolling_session_gap",
    "previous_session_time",
    "previous_session_events",
]


GROUPBY_COLS = ['user_id']

LOAD_COLS = LABEL + METADATA + OUT_FEATURE_COLUMNS

S3_BUCKET = 'dissertation-data-dmiller'
BASE_CHECK_PATH = 'lstm_experiments/checkpoints'

LSTM_CHECKPOINTS = {
    'seq_1': 'lstm_experiments/checkpoints/data_v1/n_files_30/ordinal/sequence_length_1/data_partition_None/2023_05_03_11_10/clickstream-epoch=37-loss_valid=0.59.ckpt',
    'seq_10': 'lstm_experiments/checkpoints/data_v1/n_files_30/ordinal/sequence_length_10/data_partition_None/2023_04_28_19_18/clickstream-epoch=41-loss_valid=0.59.ckpt',
    'seq_20': 'lstm_experiments/checkpoints/data_v1/n_files_30/ordinal/sequence_length_20/data_partition_None/2023_04_28_23_06/clickstream-epoch=29-loss_valid=0.59.ckpt',
    'seq_30': 'lstm_experiments/checkpoints/data_v1/n_files_30/ordinal/sequence_length_30/data_partition_None/2023_04_30_15_57/clickstream-epoch=25-loss_valid=0.59.ckpt',
    'seq_30_heuristic': 'lstm_experiments/checkpoints/data_v1/n_files_30/ordinal/sequence_length_30/data_partition_None/2023_04_30_15_57/clickstream-epoch=25-loss_valid=0.59.ckpt',
    'seq_40': 'lstm_experiments/checkpoints/data_v1/n_files_30/ordinal/sequence_length_40/data_partition_None/2023_05_01_07_09/clickstream-epoch=17-loss_valid=0.59.ckpt'
}

LSTM_CHECKPOINT_EMBEDDING = {
    'embedding_30': 'lstm_experiments/checkpoints/data_v1/n_files_30/embedded_ordinal_heuristic/sequence_length_30/data_partition_None/2023_05_10_14_18/clickstream-epoch=05-loss_valid=0.62.ckpt',
    'embedding_30_heuristic': 'lstm_experiments/checkpoints/data_v1/n_files_30/embedded_ordinal_heuristic/sequence_length_30/data_partition_None/2023_05_10_14_18/clickstream-epoch=05-loss_valid=0.62.ckpt'
}

In [6]:
# %load data_module
import pdb

import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import IterableDataset

LABEL_INDEX = 1
TOTAL_EVENTS_INDEX = 2
BATCHES = 1000000

    
class ClickstreamDataset(Dataset):
    def __init__(self, dataset_pointer_list) -> None:
        """
        Yield data in batches of BATCHES
        """
        self.events = dataset_pointer_list
        self.size = self.events[0].shape[0]


    def __getitem__(self, idx):
        events = [np.array([event[idx]]) for event in self.events]
        return np.concatenate(events, axis=1)
       

    def __len__(self):
        return self.size


In [7]:
# %load torch_model_bases
import torch 
from torch import nn
N_FEATURES = 20
class LSTMOrdinal(nn.Module):
    def __init__(self,  hidden_size=32, dropout=0.2) -> None:
        super(LSTMOrdinal, self).__init__()
        
        self.lstm = nn.LSTM(
            input_size=N_FEATURES,
            hidden_size=hidden_size,
            num_layers=2,
            batch_first=True,
            dropout=dropout
        )
        self.output = nn.Linear(
            hidden_size,
            1
        )

    def forward(self, x):


        x, _ = self.lstm(x)
        x = x[:, -1]
        return self.output(x)

In [8]:
# %load npz_extractor
import logging
import os
import zipfile

import boto3
import numpy as np
import torch
import logging

class NPZExtractor:
    logger = logging.getLogger(__name__)
    def __init__(self, input_path, n_files, n_sequences, s3_client, data_partition) -> None:
        self.input_path = input_path
        self.n_files = n_files
        self.n_sequences = n_sequences
        self.s3_client = s3_client
        self.data_partition = data_partition


    def get_dataset_pointer(self):

        read_path = os.path.join(self.input_path, f'files_used_{self.n_files}')
        if not os.path.exists(read_path):
            self.logger.info(f'Creating directory: {read_path}')
            os.makedirs(read_path)


        for _ in range(0, self.n_sequences +1, 10):
            key_zip, key_npy = (
                os.path.join(read_path, f'sequence_index_{_}.npz'),
                os.path.join(read_path, f'sequence_index_{_}')
            )
            
            self.logger.info(f'Loading pointer to dataset: {key_npy}: derived from {key_zip}')


            if not os.path.exists(key_zip):
                self.logger.info(f'Zip file to extract: {key_zip}: npy file to load: {key_npy}')
                self.s3_client.download_file(
                    'dissertation-data-dmiller',
                    key_zip,
                    key_zip
                )
            if not os.path.exists(key_npy):
                self.logger.info(f'Zip file downloaded: {key_zip}: npy file to load: {key_npy}')

                self._zip_extract(key_zip, key_npy)

        lz_concatenated_results = self._lazy_concatenate()

        if self.data_partition:
            return [p[:self.data_partition] for p in lz_concatenated_results]
        else:
            return lz_concatenated_results


    def _zip_extract(self, key_zip, key_npy):
        self.logger.info(f'Extracting file: {key_zip} -> {key_npy}')

        with zipfile.ZipFile(key_zip, 'r') as zip_ref:
            zip_ref.extractall(path=key_npy, members=['arr_0.npy'])

        self.logger.info(f'Zip file exracted: {key_zip} -> {key_npy}/arr_0.npy')

    def _lazy_concatenate(self):
        lz_concat = []
        for _ in range(0, self.n_sequences +1, 10):
            path_to_load = os.path.join(self.input_path, f'files_used_{self.n_files}', f'sequence_index_{_}', f'arr_0.npy')
            self.logger.info(f'Loading: {path_to_load}')
            lz_concat.append(np.load(path_to_load, mmap_mode='r'))
        return lz_concat

In [13]:
# %load likelihood_engagement_cpu
import torch
import os
import argparse
from torch.utils.data import DataLoader, Dataset
import logging
import boto3
from pprint import pformat
from torch import nn
import io
import tqdm
import numpy as np
import pandas as pd



pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)    
pd.set_option('mode.use_inf_as_na', True)
torch.set_printoptions(sci_mode=False, linewidth=400, precision=2)
np.set_printoptions(suppress=True, precision=4, linewidth=200)
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')

CHECKPOINT_DIR='s3://dissertation-data-dmiller/lstm_experiments/checkpoints/data_v1/n_files_30/ordinal/sequence_length_10/data_partition_None/2023_03_30_07_54'
METADATA_INDEX = 14
logger = logging.getLogger('likelihood_engagement')

def parse_args():
    parser = argparse.ArgumentParser()
    
    parser.add_argument('--n_files', type=int, default=2)
    parser.add_argument('--n_sequences', type=int, default=20)
    parser.add_argument('--file_path', type=str, default='datasets/torch_ready_data')
    parser.add_argument('--checkpoint_dir', type=str, default=CHECKPOINT_DIR)
    parser.add_argument('--write_path', type=str, default='datasets/rl_ready_data')
    parser.add_argument('--model_type', type=str, default='ordinal')
    args = parser.parse_args()
    return args

def _extract_features(tensor, n_sequences, n_features):
    
    features_dict = {}
    tensor = tensor.squeeze()
       
    metadata, features = tensor[:, :METADATA_INDEX], tensor[:, METADATA_INDEX:] 
                
    features = torch.flip(
        torch.reshape(features, (features.shape[0], 41, 20)),
        dims=[1]
    )
    
    features_dict['features_40'] = features
    features_dict['features_30'] = features[:, 10:, :]
    features_dict['features_20'] = features[:, 20:, :]
    features_dict['features_10'] = features[:, 30:, :]
    features_dict['last_sequence'] = features[:, -1, :].unsqueeze(1)
     
    return metadata, features_dict



def get_models(checkpoints: dict, s3_client, device):
    """_summary_
    Downloads models from s3 and loads them into memory.
    """
    models = {}
    for name, checkpoint in checkpoints.items():
        logger.info(f'Downloading model: {name}')
        response = s3_client.get_object(
            Bucket=S3_BUCKET,
            Key=checkpoint
        )
        buffer = io.BytesIO(response['Body'].read())
        state = torch.load(buffer, map_location=torch.device(device), )
        model = LSTMOrdinal()
        model.load_state_dict(state['state_dict'])
        model.to(device)
        models[name] = model
    return models

@torch.no_grad()
def generate_static_predictions(args):
    
    user_metadata_container = []
    client = boto3.client('s3')
    
    logger.info('Generating static prediction likelihoods for experiment')
    npz_extractor = NPZExtractor(
        args.file_path,
        args.n_files,
        args.n_sequences,
        client,
        None
           
    )
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'   
    logger.info(f'Setting device to {device}')
    
    logger.info('generating dataset pointer')
    dataset = npz_extractor.get_dataset_pointer()
    
    logger.info('Downloading model checkpoint')
    
    write_path = os.path.join(args.write_path, f'files_used_{args.n_files}')
    if not os.path.exists(write_path):
        logger.info(f'Creating directory: {write_path}')
        os.makedirs(write_path)
    
    client = boto3.client('s3')
    
    logger.info(f'Downloading models from checkpoints {LSTM_CHECKPOINTS.keys()}')
    
    models = get_models(LSTM_CHECKPOINTS, client, device)
    
    dataset = ClickstreamDataset(dataset)
    loader = DataLoader(dataset, batch_size=2048*8, shuffle=False, num_workers=8, pin_memory=True)
    activation = nn.Sigmoid()
    

    p_bar = tqdm.tqdm(loader, total=len(loader))
    
    for indx, data in enumerate(p_bar):
        p_bar.set_description(f'Processing batch: {indx}')
        data = data.to(device)
        
        metadata, features_dict = _extract_features(data, args.n_sequences + 1, 20)
        # logger.info(f'running inference on batch: {indx}')
        try:
            preds_1 = activation(models['seq_1'](features_dict['last_sequence']))
            preds_10 = activation(models['seq_10'](features_dict['features_10']))
            preds_20 = activation(models['seq_20'](features_dict['features_20']))
            preds_30 = activation(models['seq_30'](features_dict['features_30']))
            preds_40 = activation(models['seq_40'](features_dict['features_40']))
        except:
            raise Exception(f'Error processing batch: {indx}')
        
        heuristic_data = features_dict['features_30'].clone()
        preds_30_heuristic = activation(models['seq_30_heuristic'](heuristic_data))
        heuristic_scalar = torch.where(metadata[:, 4] < 25, torch.tensor(0.0), torch.tensor(1.0))
        preds_30_heuristic = preds_30_heuristic * heuristic_scalar.unsqueeze(1)

        
        # logger.info(f'Concatenating metadata and predictions')
        user_metadata = torch.cat([metadata, features_dict['last_sequence'].squeeze(), preds_1, preds_10, preds_20, preds_30, preds_30_heuristic, preds_40], dim=1)
        user_metadata_container.append(user_metadata)

   
    predicted_data = torch.cat(user_metadata_container, dim=0)
    logger.info(f'Predicted data shape: {predicted_data.shape}: generating df')
    predicted_data = pd.DataFrame(predicted_data.cpu().numpy(), columns=LABEL + METADATA + OUT_FEATURE_COLUMNS + ['seq_1', 'seq_10', 'seq_20', 'seq_30', 'seq_30_heuristic', 'seq_40'])
    
    predicted_data['date_time'] = pd.to_datetime(predicted_data[['year', 'month', 'day', 'hour', 'minute', 'second']], errors='coerce').dropna()

    logger.info('Decoding date time and sorting')
    write_path = os.path.join(write_path, 'predicted_data.parquet')
    logger.info(f'Writing to parquet: {write_path}')
    predicted_data.to_parquet(write_path)
    

In [10]:
class Arguments:
    n_files = 30
    n_sequences = 40
    file_path = 'torch_ready_data'
    write_path = 'rl_ready_data'

In [11]:
generate_static_predictions(Arguments())


2023-05-31 13:41:29,068 Found credentials in environment variables.
2023-05-31 13:41:29,129 Generating static prediction likelihoods for experiment
2023-05-31 13:41:29,185 Setting device to cuda
2023-05-31 13:41:29,186 generating dataset pointer
2023-05-31 13:41:29,187 Creating directory: torch_ready_data/files_used_30
2023-05-31 13:41:29,188 Loading pointer to dataset: torch_ready_data/files_used_30/sequence_index_0: derived from torch_ready_data/files_used_30/sequence_index_0.npz
2023-05-31 13:41:29,188 Zip file to extract: torch_ready_data/files_used_30/sequence_index_0.npz: npy file to load: torch_ready_data/files_used_30/sequence_index_0
2023-05-31 13:41:53,853 Zip file downloaded: torch_ready_data/files_used_30/sequence_index_0.npz: npy file to load: torch_ready_data/files_used_30/sequence_index_0
2023-05-31 13:41:53,854 Extracting file: torch_ready_data/files_used_30/sequence_index_0.npz -> torch_ready_data/files_used_30/sequence_index_0
2023-05-31 13:42:14,150 Zip file exracted

In [14]:
df = pd.read_parquet('rl_ready_data_2/files_used_30/predicted_data.parquet')